In [85]:
import pandas as pd
import sqlite3

# Importing smaller movie_metedata via csv
# movies = pd.read_csv("movie_metadata.csv")
# movies.head()

# read movie data table from db into dataframe
con = sqlite3.connect("db/movies.db")

sql = f"""
   SELECT * FROM movie_data
   """

movies = pd.read_sql(sql, con)
# set max display to see all available columns
pd.set_option('display.max_columns', 500)
movies.head()
# Checking for duplicates...
movies['imdb_num_code'].value_counts()

tt0360717    3
tt2224026    3
tt0232500    3
tt3332064    3
tt1976009    3
tt0077651    3
tt1243957    2
tt0318974    2
tt0138304    2
tt0413300    2
tt1298649    2
tt1528100    2
tt0929632    2
tt0275022    2
tt2467046    2
tt0380510    2
tt0327437    2
tt0365737    2
tt0119164    2
tt3040964    2
tt0087800    2
tt1051904    2
tt0467406    2
tt0080749    2
tt0157472    2
tt0795368    2
tt0120681    2
tt0218378    2
tt0765010    2
tt1234721    2
            ..
tt0360323    1
tt0212974    1
tt0295700    1
tt0391304    1
tt0331933    1
tt0323033    1
tt0118688    1
tt0113243    1
tt1591479    1
tt0282687    1
tt0186045    1
tt0083001    1
tt2205401    1
tt0423169    1
tt0947802    1
tt1695994    1
tt0119695    1
tt4438848    1
tt1268799    1
tt1114740    1
tt1351685    1
tt0100142    1
tt0097239    1
tt0144214    1
tt0480249    1
tt0264395    1
tt0264761    1
tt1658801    1
tt0266489    1
tt0482572    1
Name: imdb_num_code, Length: 3637, dtype: int64

In [28]:
# Print list of columns in smaller movies dataframe, in alphabetical order
for col in movie_data.columns.sort_values().values:
    print (col)

actor_1_facebook_likes
actor_1_name
actor_1_score
actor_2_facebook_likes
actor_2_name
actor_2_score
actor_3_facebook_likes
actor_3_name
actor_3_score
aspect_ratio
budget
cast_total_facebook_likes
color
content_rating
country
director_facebook_likes
director_name
director_score
duration
facenumber_in_poster
genres
gross
gross_margin
imdb_num_code
imdb_score
language
movie_facebook_likes
movie_imdb_link
movie_title
num_critic_for_reviews
num_user_for_reviews
num_voted_users
plot_keywords
rating_numeric
release_date
revenue
success_bins
successful
title_year


In [31]:
movies_large = pd.read_csv('movies_metadata.csv')
movies_large = movies_large.set_index('imdb_id')
movies_large.head()

C:\Users\ryanz\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,adult,belongs_to_collection,budget,genres,homepage,id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
imdb_id,,,,,,,,,,,,,,,,,,,,,,,
tt0114709,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
tt0113497,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,en,Jumanji,When siblings Judy and Peter discover an encha...,17.0155,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
tt0113228,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
tt0114885,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.85949,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
tt0113041,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.38752,/e64sOI48hQXyru7naBFyssKFxVd.jpg,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [7]:
for col in movies_large.columns.sort_values().values:
    print (col)

adult
belongs_to_collection
budget
genres
homepage
id
imdb_id
original_language
original_title
overview
popularity
poster_path
production_companies
production_countries
release_date
revenue
runtime
spoken_languages
status
tagline
title
video
vote_average
vote_count


In [30]:
movies.describe()
# Originally was ~4,900-5,100 records before re-pulling the data on 6/23 @ 8:21 pm...
# Believe it is because Seth dropped all N/A values
# New record count is not 3,738

,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_1_facebook_likes,gross,num_voted_users,cast_total_facebook_likes,facenumber_in_poster,num_user_for_reviews,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes,revenue,gross_margin,successful,rating_numeric,director_score,actor_1_score,actor_2_score,actor_3_score
count,3738.000000,3738.000000,3738.000000,3738.000000,3738.000000,3.738000e+03,3.738000e+03,3738.000000,3738.000000,3738.000000,3.738000e+03,3738.000000,3738.000000,3738.000000,3738.000000,3738.000000,3.738000e+03,3738.000000,3738.000000,3738.000000,3738.000000,3738.000000,3738.000000,3738.000000
mean,167.953986,110.289995,811.106474,773.886303,7776.301231,5.282715e+07,1.062608e+05,11565.396469,1.378812,338.186196,4.639148e+07,2002.975655,2029.302301,6.466907,2.111201,9393.966292,6.435671e+06,-48.829339,0.538523,3.218299,2.036657,3.459460,1.702410,1.393376
std,123.360761,22.632287,3075.081314,1898.369381,15548.849482,7.038027e+07,1.522432e+05,19155.840048,2.044566,411.678504,2.265329e+08,9.892475,4554.315332,1.056222,0.353487,21505.222093,2.304693e+08,1025.592962,0.498580,0.892371,1.855727,3.688152,1.525980,0.936724
min,2.000000,37.000000,0.000000,0.000000,0.000000,1.620000e+02,9.100000e+01,0.000000,0.000000,4.000000,2.180000e+02,1927.000000,0.000000,1.600000,1.180000,0.000000,-1.221330e+10,-55554.555556,0.000000,0.000000,0.441693,0.380192,0.582535,0.688765
25%,78.000000,96.000000,11.000000,196.000000,748.000000,8.515526e+06,1.986400e+04,1938.000000,0.000000,110.250000,1.000000e+07,1999.000000,387.000000,5.900000,1.850000,0.000000,-1.049825e+07,-1.048766,0.000000,3.000000,0.883387,0.760383,0.582535,0.688765
50%,140.000000,106.000000,64.000000,439.000000,1000.000000,3.024290e+07,5.410250e+04,4078.000000,1.000000,211.000000,2.500000e+07,2004.000000,690.000000,6.600000,2.350000,231.500000,1.691628e+06,0.099166,1.000000,3.000000,1.325080,1.900958,1.165069,0.688765
75%,224.000000,120.000000,240.250000,691.750000,13000.000000,6.711146e+07,1.287202e+05,16261.000000,2.000000,400.750000,5.000000e+07,2010.000000,978.500000,7.200000,2.350000,11000.000000,2.625806e+07,0.557444,1.000000,4.000000,2.650160,5.702875,2.330138,2.066294
max,813.000000,330.000000,23000.000000,23000.000000,640000.000000,7.605058e+08,1.689764e+06,656730.000000,43.000000,5060.000000,1.221550e+10,2016.000000,137000.000000,9.300000,16.000000,349000.000000,5.235058e+08,0.999861,1.000000,5.000000,11.042332,15.968051,11.650692,5.510117


In [10]:
movies_large.describe()

,revenue,runtime,vote_average,vote_count
count,4.546000e+04,45203.000000,45460.000000,45460.000000
mean,1.120935e+07,94.128199,5.618207,109.897338
std,6.433225e+07,38.407810,1.924216,491.310374
min,0.000000e+00,0.000000,0.000000,0.000000
25%,0.000000e+00,85.000000,5.000000,3.000000
50%,0.000000e+00,95.000000,6.000000,10.000000
75%,0.000000e+00,107.000000,6.800000,34.000000
max,2.787965e+09,1256.000000,10.000000,14075.000000


In [40]:
# Converting popularity from type 'object' to type 'numeric' (float)
movies['popularity'] = pd.to_numeric(movies['popularity'])
# Checking counts and types...nothing has changed.  
# Actually, there appears to be two more records now.  Why?
movies.describe()

,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_1_facebook_likes,gross,num_voted_users,cast_total_facebook_likes,facenumber_in_poster,num_user_for_reviews,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes,revenue,gross_margin,successful,rating_numeric,director_score,actor_1_score,actor_2_score,actor_3_score,popularity,vote_count,vote_average
count,3740.000000,3740.000000,3740.000000,3740.000000,3740.000000,3.740000e+03,3.740000e+03,3740.000000,3740.000000,3740.000000,3.740000e+03,3740.000000,3740.000000,3740.000000,3740.000000,3740.000000,3.740000e+03,3740.000000,3740.000000,3740.000000,3740.000000,3740.000000,3740.000000,3740.000000,3740.000000,3740.000000,3740.000000
mean,167.901070,110.281283,811.157219,773.754813,7773.212299,5.281868e+07,1.062100e+05,11561.306417,1.378075,338.055615,4.638058e+07,2002.975134,2028.531016,6.466230,2.111061,9389.410160,6.438104e+06,-48.803068,0.538770,3.217647,2.036513,3.459439,1.701811,1.392999,10.336801,904.002674,6.235401
std,123.348986,22.629368,3074.259553,1897.870108,15545.264116,7.036240e+07,1.522184e+05,19151.532615,2.044268,411.607110,2.264728e+08,9.889855,4553.219239,1.056346,0.353444,21500.372234,2.304077e+08,1025.319258,0.498561,0.892577,1.855241,3.687166,1.525792,0.936616,13.032769,1399.643328,0.876820
min,2.000000,37.000000,0.000000,0.000000,0.000000,1.620000e+02,9.100000e+01,0.000000,0.000000,4.000000,2.180000e+02,1927.000000,0.000000,1.600000,1.180000,0.000000,-1.221330e+10,-55554.555556,0.000000,0.000000,0.441693,0.380192,0.582535,0.688765,0.002404,0.000000,0.000000
25%,78.000000,96.000000,11.000000,196.000000,748.000000,8.528892e+06,1.983125e+04,1940.000000,0.000000,110.000000,1.000000e+07,1999.000000,387.000000,5.900000,1.850000,0.000000,-1.049475e+07,-1.047879,0.000000,3.000000,0.883387,0.760383,0.582535,0.688765,6.328618,122.000000,5.700000
50%,139.500000,106.000000,64.000000,439.500000,1000.000000,3.028296e+07,5.409850e+04,4073.500000,1.000000,211.000000,2.500000e+07,2004.000000,690.000000,6.600000,2.350000,232.500000,1.715416e+06,0.099840,1.000000,3.000000,1.325080,1.900958,1.165069,0.688765,9.020740,387.000000,6.300000
75%,224.000000,120.000000,241.000000,691.250000,13000.000000,6.707797e+07,1.287028e+05,16255.000000,2.000000,400.250000,5.000000e+07,2010.000000,977.500000,7.200000,2.350000,11000.000000,2.621936e+07,0.557345,1.000000,4.000000,2.650160,5.702875,2.330138,2.066294,12.160962,1042.250000,6.800000
max,813.000000,330.000000,23000.000000,23000.000000,640000.000000,7.605058e+08,1.689764e+06,656730.000000,43.000000,5060.000000,1.221550e+10,2016.000000,137000.000000,9.300000,16.000000,349000.000000,5.235058e+08,0.999861,1.000000,5.000000,11.042332,15.968051,11.650692,5.510117,547.488298,14075.000000,9.300000


In [50]:
# Create new dataframe with just the production companies and genres from large dataset
movies_large_pc = movies_large[["production_companies","genres"]]
movies_large_pc.head()

,production_companies,genres
imdb_id,,
tt0114709,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'id': 16, 'name': 'Animation'}, {'id': 35, '..."
tt0113497,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'id': 12, 'name': 'Adventure'}, {'id': 14, '..."
tt0113228,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'id': 10749, 'name': 'Romance'}, {'id': 35, ..."
tt0114885,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam..."
tt0113041,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'id': 35, 'name': 'Comedy'}]"


In [52]:
# Counting the number of production companies and genres per movie
movies_large_pc['production_companies_count'] = movies_large_pc['production_companies'].str.count('id')
movies_large_pc['genres_count'] = movies_large_pc['genres'].str.count('name')
movies_large_pc.head()

C:\Users\ryanz\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\ryanz\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,production_companies,genres,production_companies_count,genres_count
imdb_id,,,,
tt0114709,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",1.0,3
tt0113497,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",3.0,3
tt0113228,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",2.0,2
tt0114885,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",1.0,3
tt0113041,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'id': 35, 'name': 'Comedy'}]",2.0,1


In [63]:
# Add new columns to the existing movies table in sqllite db

# Columns straight from large dataset
movies = movies.merge(movies_large_pc[['production_companies_count','genres_count']], left_on="imdb_num_code", right_index=True, how="inner")
# Calculated columns based on occurrences of strings in certain columns
movies = movies.merge(movies_large[['popularity','vote_count','vote_average']], left_on="imdb_num_code", right_index=True, how="inner")
movies

# Definitions of new fields:

# Popularity: (Undetermined at this point on how this is calculated) - appears to be 0 to 100 rating scale.
# Vote Count: Number of users that voted for the movie on the TMDB website.
# Vote Average: Average score from the aforementioned votes.  
# Production Companies Count: Number of production companies that were a part of developing the movie.
# Genres Count: Number of genre classifications assigned to each movie.

# Website for large dataset:
# https://www.kaggle.com/rounakbanik/the-movies-dataset#movies_metadata.csv



,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,actor_1_name,movie_title,num_voted_users,cast_total_facebook_likes,actor_3_name,facenumber_in_poster,plot_keywords,movie_imdb_link,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes,revenue,gross_margin,successful,rating_numeric,director_score,actor_1_score,actor_2_score,actor_3_score,imdb_num_code,release_date,success_bins,production_companies_count,genres_count,popularity,vote_count,vote_average
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,CCH Pounder,Avatar,886204,4834,Wes Studi,0.0,avatar|future|marine|native|paraplegic,http://www.imdb.com/title/tt0499549/?ref_=fn_t...,3054.0,English,USA,PG-13,237000000.0,2009.0,936.0,7.9,1.78,33000,523505847.0,0.688365,1,3,3.091853,1.520767,1.747604,2.066294,tt0499549,2009-12-10,extreme success,4.0,4,185.071,12114.0,7.2
1,Color,Stephen Sommers,106.0,106.0,208.0,855.0,Jason Flemyng,3000.0,11146409.0,Action|Adventure|Horror|Sci-Fi,Djimon Hounsou,Deep Rising,26551,6558,Wes Studi,0.0,creature feature|hijacker|ocean|ship|shot in t...,http://www.imdb.com/title/tt0118956/?ref_=fn_t...,273.0,English,USA,R,45000000.0,1998.0,1000.0,6.0,2.35,0,-33853591.0,-3.037175,0,4,1.766773,3.041534,5.825346,2.066294,tt0118956,1998-01-30,no success,3.0,4,6.92246,155.0,6.0
2,Color,Terrence Malick,222.0,150.0,0.0,855.0,Michael Greyeyes,23000.0,12712093.0,Biography|Drama|History|Romance,Christian Bale,The New World,70771,27829,Wes Studi,0.0,captain|fort|love|native american|trade,http://www.imdb.com/title/tt0402399/?ref_=fn_t...,671.0,English,USA,PG-13,30000000.0,2005.0,909.0,6.7,2.35,0,-17287907.0,-1.359958,0,3,1.325080,8.364217,0.582535,2.066294,tt0402399,2005-12-25,no success,5.0,3,7.6945,336.0,6.4
3,Color,Brian Robbins,76.0,98.0,48.0,722.0,Joel David Moore,21000.0,61112916.0,Comedy|Family|Fantasy,Robert Downey Jr.,The Shaggy Dog,14888,24664,Kristin Davis,0.0,animal as human|dog movie|elevator|human becom...,http://www.imdb.com/title/tt0393735/?ref_=fn_t...,83.0,English,USA,PG,50000000.0,2006.0,936.0,4.4,2.35,747,11112916.0,0.181842,1,2,2.650160,9.884984,1.747604,2.066294,tt0393735,2006-03-09,moderate success,3.0,2,4.87891,138.0,4.5
4,Color,Brad Peyton,178.0,94.0,62.0,722.0,Dwayne Johnson,14000.0,103812241.0,Action|Adventure|Comedy|Family|Fantasy|Sci-Fi,Josh Hutcherson,Journey 2: The Mysterious Island,67296,27842,Kristin Davis,3.0,gold|hurricane|island|mountain|mysterious island,http://www.imdb.com/title/tt1397514/?ref_=fn_t...,133.0,English,USA,PG,79000000.0,2012.0,12000.0,5.8,1.85,0,24812241.0,0.239011,1,2,1.325080,2.661342,1.747604,2.066294,tt1397514,2012-01-19,average success,3.0,3,9.46307,1050.0,5.8
5,Color,Michael Patrick King,221.0,146.0,127.0,722.0,Liza Minnelli,962.0,95328937.0,Comedy|Drama|Romance,Chris Noth,Sex and the City 2,59581,4555,Kristin Davis,4.0,abu dhabi|box office hit|muslim|nanny|united a...,http://www.imdb.com/title/tt1261945/?ref_=fn_t...,293.0,English,USA,R,100000000.0,2010.0,740.0,4.3,1.85,0,-4671063.0,-0.048999,0,4,0.441693,0.760383,0.582535,2.066294,tt1261945,2010-05-26,no success,4.0,3,7.25599,434.0,5.5
6,Color,David R. Ellis,231.0,90.0,160.0,852.0,Joel David Moore,963.0,18860403.0,Horror|Thriller,Chris Zylka,Shark Night 3D,20823,4952,Katharine McPhee,0.0,lake|louisiana|redneck|shark|vacation,http://www.imdb.com/title/tt1633356/?ref_=fn_t...,130.0,English,USA,PG-13,25000000.0,2011.0,936.0,4.0,1.85,0,-6139597.0,-0.325528,0,3,2.650160,0.380192,1.747604,0.688765,tt1633356,2011-09-02,no success,1.0,2,6.06243,134.0,4.9
7,Color,Peter Hyams,174.0,121.0,0.0,595.0,Mark Margolis,1000.0,66862068.0,Action|Fantasy|Horror|Mystery,CCH Pounder,End of Days,89509,3903,Udo Kier,0.0,atheist|ex cop|female masturbation|female nudi...,http://www.imdb.com/title/tt0146675/?ref_=fn_t...,524.0,English,USA,R,830

In [60]:
# Code to drop dups if the above cell is written multiple times
# movies = movies.drop(['popularity_x','vote_count_x','vote_average_x','production_companies_count_x',
#                     'genres_count_x','production_companies_count_y','genres_count_y','popularity_y',
#                     'vote_count_y','vote_average_y'],axis=1)


In [82]:
# Seem to be duplicate records now...
movies['imdb_num_code'].value_counts()

# Also, why is there 128 duplicate records for tt0157472?

tt0157472    128
tt0360717      3
tt1976009      3
tt0232500      3
tt3332064      3
tt0077651      3
tt2224026      3
tt1243957      2
tt0318974      2
tt0138304      2
tt0413300      2
tt1298649      2
tt1528100      2
tt0929632      2
tt0275022      2
tt2467046      2
tt0380510      2
tt0327437      2
tt0365737      2
tt0119164      2
tt3040964      2
tt0087800      2
tt1051904      2
tt0467406      2
tt0080749      2
tt1291150      2
tt0795368      2
tt0120681      2
tt0218378      2
tt0765010      2
            ... 
tt0360323      1
tt0212974      1
tt0295700      1
tt0391304      1
tt0331933      1
tt0323033      1
tt0118688      1
tt0113243      1
tt1591479      1
tt0282687      1
tt0186045      1
tt0083001      1
tt2205401      1
tt0423169      1
tt0947802      1
tt1695994      1
tt0119695      1
tt4438848      1
tt1268799      1
tt1114740      1
tt1351685      1
tt0100142      1
tt0097239      1
tt0144214      1
tt0480249      1
tt0264395      1
tt0264761      1
tt1658801     

In [94]:
# Remove duplicates
# removeDups2_movies = movies.drop_duplicates(subset ="imdb_num_code", 
#                      keep = 'first', inplace = True) 

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,actor_1_name,movie_title,num_voted_users,cast_total_facebook_likes,actor_3_name,facenumber_in_poster,plot_keywords,movie_imdb_link,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes,revenue,gross_margin,successful,rating_numeric,director_score,actor_1_score,actor_2_score,actor_3_score,imdb_num_code,release_date,success_bins
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,CCH Pounder,Avatar,886204,4834,Wes Studi,0.0,avatar|future|marine|native|paraplegic,http://www.imdb.com/title/tt0499549/?ref_=fn_t...,3054.0,English,USA,PG-13,237000000.0,2009.0,936.0,7.9,1.78,33000,523505847.0,0.688365,1,3,3.091853,1.520767,1.747604,2.066294,tt0499549,2009-12-10,extreme success
1,Color,Stephen Sommers,106.0,106.0,208.0,855.0,Jason Flemyng,3000.0,11146409.0,Action|Adventure|Horror|Sci-Fi,Djimon Hounsou,Deep Rising,26551,6558,Wes Studi,0.0,creature feature|hijacker|ocean|ship|shot in t...,http://www.imdb.com/title/tt0118956/?ref_=fn_t...,273.0,English,USA,R,45000000.0,1998.0,1000.0,6.0,2.35,0,-33853591.0,-3.037175,0,4,1.766773,3.041534,5.825346,2.066294,tt0118956,1998-01-30,no success
2,Color,Terrence Malick,222.0,150.0,0.0,855.0,Michael Greyeyes,23000.0,12712093.0,Biography|Drama|History|Romance,Christian Bale,The New World,70771,27829,Wes Studi,0.0,captain|fort|love|native american|trade,http://www.imdb.com/title/tt0402399/?ref_=fn_t...,671.0,English,USA,PG-13,30000000.0,2005.0,909.0,6.7,2.35,0,-17287907.0,-1.359958,0,3,1.325080,8.364217,0.582535,2.066294,tt0402399,2005-12-25,no success
3,Color,Brian Robbins,76.0,98.0,48.0,722.0,Joel David Moore,21000.0,61112916.0,Comedy|Family|Fantasy,Robert Downey Jr.,The Shaggy Dog,14888,24664,Kristin Davis,0.0,animal as human|dog movie|elevator|human becom...,http://www.imdb.com/title/tt0393735/?ref_=fn_t...,83.0,English,USA,PG,50000000.0,2006.0,936.0,4.4,2.35,747,11112916.0,0.181842,1,2,2.650160,9.884984,1.747604,2.066294,tt0393735,2006-03-09,moderate success
4,Color,Brad Peyton,178.0,94.0,62.0,722.0,Dwayne Johnson,14000.0,103812241.0,Action|Adventure|Comedy|Family|Fantasy|Sci-Fi,Josh Hutcherson,Journey 2: The Mysterious Island,67296,27842,Kristin Davis,3.0,gold|hurricane|island|mountain|mysterious island,http://www.imdb.com/title/tt1397514/?ref_=fn_t...,133.0,English,USA,PG,79000000.0,2012.0,12000.0,5.8,1.85,0,24812241.0,0.239011,1,2,1.325080,2.661342,1.747604,2.066294,tt1397514,2012-01-19,average success
5,Color,Michael Patrick King,221.0,146.0,127.0,722.0,Liza Minnelli,962.0,95328937.0,Comedy|Drama|Romance,Chris Noth,Sex and the City 2,59581,4555,Kristin Davis,4.0,abu dhabi|box office hit|muslim|nanny|united a...,http://www.imdb.com/title/tt1261945/?ref_=fn_t...,293.0,English,USA,R,100000000.0,2010.0,740.0,4.3,1.85,0,-4671063.0,-0.048999,0,4,0.441693,0.760383,0.582535,2.066294,tt1261945,2010-05-26,no success
6,Color,David R. Ellis,231.0,90.0,160.0,852.0,Joel David Moore,963.0,18860403.0,Horror|Thriller,Chris Zylka,Shark Night 3D,20823,4952,Katharine McPhee,0.0,lake|louisiana|redneck|shark|vacation,http://www.imdb.com/title/tt1633356/?ref_=fn_t...,130.0,English,USA,PG-13,25000000.0,2011.0,936.0,4.0,1.85,0,-6139597.0,-0.325528,0,3,2.650160,0.380192,1.747604,0.688765,tt1633356,2011-09-02,no success
7,Color,Peter Hyams,174.0,121.0,0.0,595.0,Mark Margolis,1000.0,66862068.0,Action|Fantasy|Horror|Mystery,CCH Pounder,End of Days,89509,3903,Udo Kier,0.0,atheist|ex cop|female masturbation|female nudi...,http://www.imdb.com/title/tt0146675/?ref_=fn_t...,524.0,English,USA,R,83000000.0,1999.0,1000.0,5.7,2.35,2000,-16137932.0,-0.241362,0,4,2.208466,1.520767,2.912673,2.066294,tt0146675,1999-11-24,no success
8,Color,William Malone,104.0,101.0,37.0,595.0,Jeffrey Combs,2000.0,13208023.0,Crime|Horror|Thriller,Natascha McElho

In [95]:
movies

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,actor_1_name,movie_title,num_voted_users,cast_total_facebook_likes,actor_3_name,facenumber_in_poster,plot_keywords,movie_imdb_link,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes,revenue,gross_margin,successful,rating_numeric,director_score,actor_1_score,actor_2_score,actor_3_score,imdb_num_code,release_date,success_bins
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,CCH Pounder,Avatar,886204,4834,Wes Studi,0.0,avatar|future|marine|native|paraplegic,http://www.imdb.com/title/tt0499549/?ref_=fn_t...,3054.0,English,USA,PG-13,237000000.0,2009.0,936.0,7.9,1.78,33000,523505847.0,0.688365,1,3,3.091853,1.520767,1.747604,2.066294,tt0499549,2009-12-10,extreme success
1,Color,Stephen Sommers,106.0,106.0,208.0,855.0,Jason Flemyng,3000.0,11146409.0,Action|Adventure|Horror|Sci-Fi,Djimon Hounsou,Deep Rising,26551,6558,Wes Studi,0.0,creature feature|hijacker|ocean|ship|shot in t...,http://www.imdb.com/title/tt0118956/?ref_=fn_t...,273.0,English,USA,R,45000000.0,1998.0,1000.0,6.0,2.35,0,-33853591.0,-3.037175,0,4,1.766773,3.041534,5.825346,2.066294,tt0118956,1998-01-30,no success
2,Color,Terrence Malick,222.0,150.0,0.0,855.0,Michael Greyeyes,23000.0,12712093.0,Biography|Drama|History|Romance,Christian Bale,The New World,70771,27829,Wes Studi,0.0,captain|fort|love|native american|trade,http://www.imdb.com/title/tt0402399/?ref_=fn_t...,671.0,English,USA,PG-13,30000000.0,2005.0,909.0,6.7,2.35,0,-17287907.0,-1.359958,0,3,1.325080,8.364217,0.582535,2.066294,tt0402399,2005-12-25,no success
3,Color,Brian Robbins,76.0,98.0,48.0,722.0,Joel David Moore,21000.0,61112916.0,Comedy|Family|Fantasy,Robert Downey Jr.,The Shaggy Dog,14888,24664,Kristin Davis,0.0,animal as human|dog movie|elevator|human becom...,http://www.imdb.com/title/tt0393735/?ref_=fn_t...,83.0,English,USA,PG,50000000.0,2006.0,936.0,4.4,2.35,747,11112916.0,0.181842,1,2,2.650160,9.884984,1.747604,2.066294,tt0393735,2006-03-09,moderate success
4,Color,Brad Peyton,178.0,94.0,62.0,722.0,Dwayne Johnson,14000.0,103812241.0,Action|Adventure|Comedy|Family|Fantasy|Sci-Fi,Josh Hutcherson,Journey 2: The Mysterious Island,67296,27842,Kristin Davis,3.0,gold|hurricane|island|mountain|mysterious island,http://www.imdb.com/title/tt1397514/?ref_=fn_t...,133.0,English,USA,PG,79000000.0,2012.0,12000.0,5.8,1.85,0,24812241.0,0.239011,1,2,1.325080,2.661342,1.747604,2.066294,tt1397514,2012-01-19,average success
5,Color,Michael Patrick King,221.0,146.0,127.0,722.0,Liza Minnelli,962.0,95328937.0,Comedy|Drama|Romance,Chris Noth,Sex and the City 2,59581,4555,Kristin Davis,4.0,abu dhabi|box office hit|muslim|nanny|united a...,http://www.imdb.com/title/tt1261945/?ref_=fn_t...,293.0,English,USA,R,100000000.0,2010.0,740.0,4.3,1.85,0,-4671063.0,-0.048999,0,4,0.441693,0.760383,0.582535,2.066294,tt1261945,2010-05-26,no success
6,Color,David R. Ellis,231.0,90.0,160.0,852.0,Joel David Moore,963.0,18860403.0,Horror|Thriller,Chris Zylka,Shark Night 3D,20823,4952,Katharine McPhee,0.0,lake|louisiana|redneck|shark|vacation,http://www.imdb.com/title/tt1633356/?ref_=fn_t...,130.0,English,USA,PG-13,25000000.0,2011.0,936.0,4.0,1.85,0,-6139597.0,-0.325528,0,3,2.650160,0.380192,1.747604,0.688765,tt1633356,2011-09-02,no success
7,Color,Peter Hyams,174.0,121.0,0.0,595.0,Mark Margolis,1000.0,66862068.0,Action|Fantasy|Horror|Mystery,CCH Pounder,End of Days,89509,3903,Udo Kier,0.0,atheist|ex cop|female masturbation|female nudi...,http://www.imdb.com/title/tt0146675/?ref_=fn_t...,524.0,English,USA,R,83000000.0,1999.0,1000.0,5.7,2.35,2000,-16137932.0,-0.241362,0,4,2.208466,1.520767,2.912673,2.066294,tt0146675,1999-11-24,no success
8,Color,William Malone,104.0,101.0,37.0,595.0,Jeffrey Combs,2000.0,13208023.0,Crime|Horror|Thriller,Natascha McElho